# Hi-C Scaffolding MynaBird

In [2]:
#Set Up
WKDIR=/workspace/hraijc/HiC_trials/HiC23/NovaSeq/Myna

mkdir -p $WKDIR/log
cd $WKDIR

In [4]:
#IN INTERACTIVE SESSION:
#Use Srun not Salloc
"""
srun --time 01:00:00 --cpus-per-task 1 --mem-per-cpu 4G --pty bash
#WKDIR=/workspace/hraijc/HiC_trials/HiC23/NovaSeq/Urchin
cd $WKDIR
git clone git@github.com:PlantandFoodResearch/hic-scaffolding-pipeline.git
cd ${WKDIR}/hic-scaffolding-pipeline/
git checkout dev
cd ${WKDIR}/hic-scaffolding-pipeline/containers
sudo -E singularity build pipeline_container.sif pipeline_container.def

"""

In [1]:
#Combine Reads from flowcells

sbatch << EOF
#!/bin/bash
#SBATCH -J cat
#SBATCH --cpus-per-task=1
#SBATCH --mem=1G
#SBATCH --time=1:00:00
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err



cd /input/genomic/plant/Actinidia/chinensis/ExperimentRequestor11022/AGRF_CAGRF230414403_HFFHYDSX7
date
cat MBC3_C_HFFHYDSX7_CAGGCG_L002_R1.fastq.gz MBC3_C_HFFHYDSX7_CAGGCG_L003_R1.fastq.gz > /workspace/hraijc/TEMP/Myna_HiC23_R1.fastq.gz
cat MBC3_C_HFFHYDSX7_CAGGCG_L002_R2.fastq.gz MBC3_C_HFFHYDSX7_CAGGCG_L003_R2.fastq.gz > /workspace/hraijc/TEMP/Myna_HiC23_R2.fastq.gz
date




EOF


Submitted batch job 1491010


In [ ]:
# Modify Config file

cat ${WKDIR}/hic-scaffolding-pipeline/nextflow.config

In [8]:
# Run

sbatch << EOF
#!/bin/bash -e

#SBATCH -J hic_pipeline_${USER}
#SBATCH --time=167:00:00
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --mem=4G

ml nextflow/22.10.4
ml apptainer/1.1

cd ${WKDIR}/hic-scaffolding-pipeline

srun nextflow main.nf -resume
EOF





Submitted batch job 1941063


In [ ]:
APREFIX=Atri_hic1

In [ ]:
#Remove PCR duplicates to speed up analysis.
sbatch << EOF
#!/bin/bash
#SBATCH -J hic
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=12
#SBATCH --mem=10G
#SBATCH --time=15:00:00

module load samtools/1.16

APREFIX=Atri_hic1


cd /workspace/hraijc/HiC_trials/HiC23/NovaSeq/Myna/hic-scaffolding-pipeline/work/d7/9f9231cf5d6ad9d69cf958c935a037/

samtools view -S -b -h -@ 12 -F 2316 ${APREFIX}_marked_byread.sam > ${APREFIX}_presort_marked.bam

samtools sort -@ 12 ${APREFIX}_presort_marked.bam -o ${APREFIX}_marked.bam

samtools view -h -@ 12 -F 3340 ${APREFIX}_marked.bam > ${APREFIX}_dedup.sam 







EOF


In [1]:
seff 1940533

Job ID: 1940533
Cluster: powerplant
User/Group: hraijc/hraijc
State: COMPLETED (exit code 0)
Nodes: 1
Cores per node: 12
CPU Utilized: 2-22:46:24
CPU Efficiency: 54.93% of 5-08:50:00 core-walltime
Job Wall-clock time: 10:44:10
Memory Utilized: 9.85 GB
Memory Efficiency: 98.52% of 10.00 GB


In [7]:
#Remove PCR duplicates to speed up analysis.
sbatch << EOF
#!/bin/bash
#SBATCH -J hic
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=12
#SBATCH --mem=10G
#SBATCH --time=15:00:00

module load samtools/1.16

APREFIX=Atri_hic1


cd /powerplant/workspace/hraijc/HiC_trials/HiC23/NovaSeq/Myna/hic-scaffolding-pipeline/TEMP

samtools sort -n -@ 12 Atri_hic1_dedup_namesort.sam -o Atri_hic1_dedup.sam




EOF


Submitted batch job 1940914


In [2]:
seff 1940914

Job ID: 1940914
Cluster: powerplant
User/Group: hraijc/hraijc
State: COMPLETED (exit code 0)
Nodes: 1
Cores per node: 12
CPU Utilized: 03:17:39
CPU Efficiency: 15.44% of 21:19:48 core-walltime
Job Wall-clock time: 01:46:39
Memory Utilized: 10.11 GB
Memory Efficiency: 101.11% of 10.00 GB


In [3]:
#Remove PCR duplicates to speed up analysis.
sbatch << EOF
#!/bin/bash
#SBATCH -J hic
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=6
#SBATCH --mem=1G
#SBATCH --time=15:00:00

module load samtools/1.16


cd /workspace/hraijc/HiC_trials/HiC23/NovaSeq/Myna/hic-scaffolding-pipeline/results/bam

samtools view -b -h -@ 6 -F 3340 Atri_hic1_yahs_predup.bam > Atri_hic1_yahs_dedup.bam



EOF


Submitted batch job 1944752


## YAHS

In [19]:
#Set Up
WKDIR=/workspace/hraijc/HiC_trials/HiC23/NovaSeq/Myna/yahs
ASSEMBLY=/workspace/hraijc/HiC_trials/HiC23/Assemblies/Atri_polished_final_step3.purge.vecscreen.trim.filtered.fna
BAMFILE=/workspace/hraijc/HiC_trials/HiC23/NovaSeq/Myna/hic-scaffolding-pipeline/results/bam/Atri_hic1_dedup.bam
OUT_PREFIX=Atri_hic1
mkdir -p $WKDIR/log
cd $WKDIR

In [14]:
## Just run YAHS

sbatch << EOF
#!/bin/bash
#SBATCH -J YAHS
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=1
#SBATCH --mem=12G
#SBATCH --time=23:00:00

cd ${WKDIR}

/workspace/hraijc/git_clones/yahs/yahs ${ASSEMBLY} ${BAMFILE} --no-mem-check -o ${OUT_PREFIX}_yahs -r 20000,50000,100000,200000,500000,1000000,2000000,5000000,10000000,20000000,50000000,100000000,200000000,500000000


EOF


Submitted batch job 1948518


In [17]:
## Index new assembly.

sbatch << EOF
#!/bin/bash
#SBATCH -J YAHS
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=1
#SBATCH --mem=1G
#SBATCH --time=1:00:00

cd ${WKDIR}

module load samtools
samtools faidx Atri_hic1_yahs_scaffolds_final.fa

EOF


Submitted batch job 1949004


In [18]:
# Make heatmap of YAHS output
cd ${WKDIR}
out=Atri_hic1_yahs # prefix of outfiles produced by YAHS.  
contigs=${ASSEMBLY} # need to be indexed, i.e., test.fa.gz.fai in same directory
wkdir=${WKDIR}


sbatch --job-name=hic_mapyahs \
    -o ${WKDIR}/log/%J.out \
    -e ${WKDIR}/log/%J.err \
    --cpus-per-task=8 \
    --mem=12G \
    --time=03:10:00 \
    --export=out=$out,contigs=$contigs,wkdir=$wkdir \
    /workspace/hraijc/Gitrepos/High-quality-genomes/Methods/DNase_HiC/notebooks/yahs_contactmapgen2.sh

Submitted batch job 1949005


In [20]:
##############
#Do manual scaffolding cleanup in JuiceBox then run

##############


sbatch << EOF
#!/bin/bash
#SBATCH -J YAHS
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=1
#SBATCH --mem=4G
#SBATCH --time=2:00:00

#module load samtools


cd ${WKDIR}



/workspace/hraijc/git_clones/yahs/juicer post -o Atri_hic1_yahs2 Atri_hic1_yahs_JBAT.review.assembly Atri_hic1_yahs_JBAT.liftover.agp ${ASSEMBLY}



EOF


Submitted batch job 1949015


### Subset unique mapped reads

In [32]:
WKDIR=/workspace/hraijc/HiC_trials/HiC23/NovaSeq/Myna/hic-scaffolding-pipeline/results
BAMFILE=${WKDIR}/bam/Atri_hic1_yahs_dedup.bam
mkdir -p ${WKDIR}/log

In [33]:
##############
# Subset HiC reads to keep unique mapped reads only. speed up analysis for future iterations.

##############


sbatch << EOF
#!/bin/bash
#SBATCH -J YAHS
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=8
#SBATCH --mem=12G
#SBATCH --time=2:00:00

cd ${WKDIR}

module load samtools 

samtools sort -@ 8 -n ${BAMFILE} | samtools fastq -@ 8 ${BAMFILE} -1 Atri_hic1_umap_R1.fastq.gz -2 Atri_hic1_umap_R2.fastq.gz -0 /dev/null -s /dev/null -n



EOF

Submitted batch job 1949203


## ASSEMBLYQC

In [ ]:
#git clone git@github.com:PlantandFoodResearch/assembly_qc.git


In [26]:
#Set Up
WKDIR=/workspace/hraijc/HiC_trials/HiC23/NovaSeq/Myna/assembly_qc
ASSEMBLY=/workspace/hraijc/HiC_trials/HiC23/NovaSeq/Myna/Atri_hic1_yahs2.fa
mkdir -p $WKDIR/log
cd $WKDIR

In [30]:
cat nextflow.config | grep -v "//"

includeConfig './conf/base.config'

params {
    
    target_assemblies       = [
        ["Atri_hic1_yahs2", "/workspace/hraijc/HiC_trials/HiC23/NovaSeq/Myna/Atri_hic1_yahs2.fa"]
    ]

    assembly_gff3           = [
        ]
    
    assemblathon_stats {
        n_limit             = 100
    }

    ncbi_fcs_adaptor {


    }

    ncbi_fcs_gx {
        skip                = 1

        tax_id              = "78410"

        db_path             = "/workspace/ComparativeDataSources/NCBI/FCS/GX/r2023-01-24"

    }
    
    busco {
        skip                = 0

        mode                = "geno"
        
        lineage_datasets    = ["aves_odb10"]

        download_path       = "/workspace/ComparativeDataSources/BUSCO/assembly_qc"
    }

    tidk {
        skip                = 0
        
        repeat_seq          = "TTAGGG"

        filter_by_size      = 0

        filter_size_bp      = 1000000
    }

    lai {
        skip                = 1

        mode                = ""

 

In [ ]:
sbatch ./assembly_qc_pfr.sh
